<a href="https://colab.research.google.com/github/dalatorrem/Diplomado_Metodos_UCentral_2023/blob/main/28_practica_analisis_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo de predicción del puntaje en la prueba saber 11

Contexto Analítico: construir un modelo lineal que teniendo como variables de entrada variables sociodemográficas nos permita dar una predicción del puntaje global.


In [50]:
import pandas                   as pd
import numpy                    as np
import matplotlib.pyplot        as plt
import seaborn                  as sns
from   scipy                    import stats
import statsmodels.formula.api  as smf
import scipy
import warnings
from sklearn.model_selection    import train_test_split
from sklearn.linear_model       import LinearRegression
from sklearn.metrics            import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics            import mean_absolute_percentage_error
import plotly.express as px

In [8]:
path_data = 'https://raw.githubusercontent.com/dalatorrem/Diplomado_Metodos_UCentral_2023/main/data/Resultados__nicos_Saber_11_sample.csv'
data_saber = pd.read_csv(path_data)
data_saber

,Unnamed: 0,PERIODO,ESTU_TIPODOCUMENTO,ESTU_CONSECUTIVO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,...,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,DESEMP_INGLES,PUNT_INGLES,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_LECTURA_CRITICA,PUNT_GLOBAL
0,0,20131,CR,SB11201310000414,URBANO,N,B,ACADÉMICO,3.118480e+11,3.118480e+11,...,Si,Si,Si,B+,94.0,88.0,NaN,NaN,NaN,NaN
1,1,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,1.410160e+11,2.410160e+11,...,Si,Si,Si,B1,71.0,66.0,70.0,65.0,69.0,339.0
2,2,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,1.410160e+11,2.410160e+11,...,Si,Si,Si,B1,71.0,66.0,70.0,65.0,69.0,339.0
3,3,20122,TI,SB11201220204399,URBANO,N,A,TÉCNICO/ACADÉMICO,1.631300e+11,1.631300e+11,...,Si,Si,No,A1,48.0,45.0,NaN,NaN,NaN,NaN
4,4,20132,TI,SB11201320464198,URBANO,N,A,TÉCNICO,1.190010e+11,1.190010e+11,...,Si,Si,Si,A-,43.0,52.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,20162,TI,SB11201620287491,RURAL,NaN,A,ACADÉMICO,2.687200e+11,2.687200e+11,...,No,No,No,A1,55.0,50.0,57.0,48.0,53.0,262.0
9996,9996,20132,TI,SB11201320276289,URBANO,N,A,TÉCNICO/ACADÉMICO,2.470010e+11,2.470010e+11,...,Si,Si,No,A-,39.0,57.0,NaN,NaN,NaN,NaN
9997,9997,20194,TI,SB11201940202131,RURAL,N,A,ACADÉMICO,2.735040e+11,2.735040e+11,...,No,No,No,A-,36.0,36.0,52.0,51.0,55.0,238.0
9998,9998,20194,TI,SB11201940202131,RURAL,N,A,ACADÉMICO,2.735040e+11,2.735040e+11,...,No,No,No,A-,36.0,36.0,52.0,51.0,55.0,238.0


In [9]:
data_saber.shape

(10000, 52)

In [10]:
TARGET = 'PUNT_GLOBAL'

# OJO con las variables a usar

In [11]:
data_saber.columns

Index(['Unnamed: 0', 'PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA

In [12]:
data_modelo_1 = data_saber[['PUNT_INGLES', 'PUNT_MATEMATICAS',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA',
       'PUNT_GLOBAL']].copy()
data_modelo_1 = data_modelo_1.dropna()
data_modelo_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5619 entries, 1 to 9999
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PUNT_INGLES               5619 non-null   float64
 1   PUNT_MATEMATICAS          5619 non-null   float64
 2   PUNT_SOCIALES_CIUDADANAS  5619 non-null   float64
 3   PUNT_C_NATURALES          5619 non-null   float64
 4   PUNT_LECTURA_CRITICA      5619 non-null   float64
 5   PUNT_GLOBAL               5619 non-null   float64
dtypes: float64(6)
memory usage: 307.3 KB


In [13]:
formula_1 = f'{TARGET} ~ PUNT_INGLES + PUNT_MATEMATICAS + PUNT_SOCIALES_CIUDADANAS + PUNT_C_NATURALES + PUNT_LECTURA_CRITICA'
model_no_usable = smf.ols(formula=formula_1,data=data_modelo_1).fit()
print(model_no_usable.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.676e+07
Date:                Wed, 30 Aug 2023   Prob (F-statistic):               0.00
Time:                        23:21:41   Log-Likelihood:                -2887.6
No. Observations:                5619   AIC:                             5787.
Df Residuals:                    5613   BIC:                             5827.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

**Este modelo no aporta por dos razones:**

1. La variable de salida está en función de las variables de entrada, especificamente se sabe que el puntaje global es una combinación de los demás puntajes.
2. Los puntajes por componente aparecen al tiempo del puntaje global

# OJo con los códigos

In [14]:
data_saber.columns

Index(['Unnamed: 0', 'PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA

In [15]:
data_modelo_2 = data_saber[['ESTU_CONSECUTIVO', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE', 'COLE_COD_DEPTO_UBICACION',
       'PUNT_GLOBAL']].copy()
data_modelo_2 = data_modelo_2.dropna()
data_modelo_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5619 entries, 1 to 9999
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ESTU_CONSECUTIVO               5619 non-null   object 
 1   COLE_COD_DANE_ESTABLECIMIENTO  5619 non-null   float64
 2   COLE_COD_DANE_SEDE             5619 non-null   float64
 3   COLE_COD_DEPTO_UBICACION       5619 non-null   int64  
 4   PUNT_GLOBAL                    5619 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 263.4+ KB


In [16]:
formula_2 = f'{TARGET} ~ ESTU_CONSECUTIVO + COLE_COD_DANE_ESTABLECIMIENTO + COLE_COD_DANE_SEDE + COLE_COD_DEPTO_UBICACION'
model_no_usable = smf.ols(formula=formula_2,data=data_modelo_2).fit()
print(model_no_usable.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.348e+09
Date:                Wed, 30 Aug 2023   Prob (F-statistic):               0.00
Time:                        23:24:51   Log-Likelihood:                 9560.8
No. Observations:                5619   AIC:                        -1.911e+04
Df Residuals:                    5615   BIC:                        -1.909e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


**Las variables COLE_COD_DANE_ESTABLECIMIENTO + COLE_COD_DANE_SEDE + COLE_COD_DEPTO_UBICACION son variables categóricos y deben ser tratadas de esa forma**

**La columna ESTU_CONSECUTIVO no es una variable, es un código de identificación, por lo tanto seguramente es único**

# Selección de variables de acuerdo al conocimiento del equipo que modela:

In [17]:
list_vars_model = ['Unnamed: 0', 'PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSONASHOGAR', 'FAMI_TIENEAUTOMOVIL',
       'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       'DESEMP_INGLES', 'PUNT_INGLES', 'PUNT_MATEMATICAS',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA',
       'PUNT_GLOBAL']

In [18]:
data_saber['COLE_NATURALEZA'].unique()

array(['NO OFICIAL', 'OFICIAL'], dtype=object)

Resultados 'DESEMP_INGLES', 'PUNT_INGLES', 'PUNT_MATEMATICAS',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA',
       'PUNT_GLOBAL'

No es variable, es constante ESTU_ESTUDIANTE

Error al guardar 'Unnamed: 0'

PERIODO no se utilizará para este análisis

Identificador, no es variable ESTU_CONSECUTIVO

In [19]:
list_vars_model = [ 'ESTU_TIPODOCUMENTO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSONASHOGAR', 'FAMI_TIENEAUTOMOVIL',
       'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       ]

In [20]:
list_cols = list_vars_model + [TARGET]
data_modelo_3 = data_saber[list_cols].dropna().copy()
data_modelo_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4611 entries, 1 to 9999
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ESTU_TIPODOCUMENTO             4611 non-null   object 
 1   COLE_AREA_UBICACION            4611 non-null   object 
 2   COLE_BILINGUE                  4611 non-null   object 
 3   COLE_CALENDARIO                4611 non-null   object 
 4   COLE_CARACTER                  4611 non-null   object 
 5   COLE_COD_DANE_ESTABLECIMIENTO  4611 non-null   float64
 6   COLE_COD_DANE_SEDE             4611 non-null   float64
 7   COLE_COD_DEPTO_UBICACION       4611 non-null   int64  
 8   COLE_COD_MCPIO_UBICACION       4611 non-null   int64  
 9   COLE_CODIGO_ICFES              4611 non-null   float64
 10  COLE_DEPTO_UBICACION           4611 non-null   object 
 11  COLE_GENERO                    4611 non-null   object 
 12  COLE_JORNADA                   4611 non-null   o

In [21]:
data_modelo_3['COLE_AREA_UBICACION'].unique()

array(['RURAL', 'URBANO'], dtype=object)

## No realizar el análisis de este modo porque variables con mayor número de categorias pueden dar un r2 más alto sin ser buenas predictoras

In [22]:
data_modelo_3[list_vars_model] = data_modelo_3[list_vars_model].astype('str')
data_modelo_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4611 entries, 1 to 9999
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ESTU_TIPODOCUMENTO             4611 non-null   object 
 1   COLE_AREA_UBICACION            4611 non-null   object 
 2   COLE_BILINGUE                  4611 non-null   object 
 3   COLE_CALENDARIO                4611 non-null   object 
 4   COLE_CARACTER                  4611 non-null   object 
 5   COLE_COD_DANE_ESTABLECIMIENTO  4611 non-null   object 
 6   COLE_COD_DANE_SEDE             4611 non-null   object 
 7   COLE_COD_DEPTO_UBICACION       4611 non-null   object 
 8   COLE_COD_MCPIO_UBICACION       4611 non-null   object 
 9   COLE_CODIGO_ICFES              4611 non-null   object 
 10  COLE_DEPTO_UBICACION           4611 non-null   object 
 11  COLE_GENERO                    4611 non-null   object 
 12  COLE_JORNADA                   4611 non-null   o

In [23]:
for col in list_vars_model:
  model_col = smf.ols(formula=f'{TARGET}~{col}',data=data_modelo_3).fit()
  print(f'Modelo con sólo la variable {col} tiene un R2 de : {round(model_col.rsquared, 3)}')

Modelo con sólo la variable ESTU_TIPODOCUMENTO tiene un R2 de : 0.057
Modelo con sólo la variable COLE_AREA_UBICACION tiene un R2 de : 0.01
Modelo con sólo la variable COLE_BILINGUE tiene un R2 de : 0.021
Modelo con sólo la variable COLE_CALENDARIO tiene un R2 de : 0.057
Modelo con sólo la variable COLE_CARACTER tiene un R2 de : 0.007
Modelo con sólo la variable COLE_COD_DANE_ESTABLECIMIENTO tiene un R2 de : 0.798
Modelo con sólo la variable COLE_COD_DANE_SEDE tiene un R2 de : 0.803
Modelo con sólo la variable COLE_COD_DEPTO_UBICACION tiene un R2 de : 0.073
Modelo con sólo la variable COLE_COD_MCPIO_UBICACION tiene un R2 de : 0.259
Modelo con sólo la variable COLE_CODIGO_ICFES tiene un R2 de : 0.865
Modelo con sólo la variable COLE_DEPTO_UBICACION tiene un R2 de : 0.074
Modelo con sólo la variable COLE_GENERO tiene un R2 de : 0.023
Modelo con sólo la variable COLE_JORNADA tiene un R2 de : 0.141
Modelo con sólo la variable COLE_MCPIO_UBICACION tiene un R2 de : 0.27
Modelo con sólo la va

# Recomendable Train-test split

Lo recomendable en el momento en que empieza a modelar (cuando tiene definido las variables candidatas haga el train - test split)

test_size: valor entre el 20% y el 30%

In [40]:
r_state = 20230830

In [47]:
df_train, df_test = train_test_split(data_modelo_3, test_size=0.25, random_state=r_state)
len(df_train), len(df_test), len(df_train)/len(data_modelo_3), len(df_test)/len(data_modelo_3)

(3458, 1153, 0.7499457818260681, 0.2500542181739319)

In [48]:
df_train['PUNT_GLOBAL'].mean()

254.86524002313476

Mismo ejercicio de celdas anteriores pero ajustando con los de train y revisando desempeño (R2) en train y test.

In [53]:
model_col.predict(df_train)

4468    258.933144
1955    251.233542
8976    251.233542
3711    251.233542
3473    251.233542
           ...    
8391    251.233542
5841    249.466501
6014    249.466501
8083    258.933144
9688    249.466501
Length: 3458, dtype: float64

In [54]:
df_train[TARGET]

4468    246.0
1955    236.0
8976    256.0
3711    223.0
3473    280.0
        ...  
8391    232.0
5841    275.0
6014    259.0
8083    366.0
9688    306.0
Name: PUNT_GLOBAL, Length: 3458, dtype: float64

In [57]:
r2_score(df_train[TARGET], model_col.predict(df_train))

0.008147200560166268

In [58]:
model_col.rsquared

0.008147200560166157

In [66]:
df_train_n = df_train.copy()
df_test_n = df_test.copy()

In [68]:
len(df_train_n['COLE_COD_DANE_ESTABLECIMIENTO'].unique())

2206

In [69]:
for col in list_vars_model:
  list_rep = list(set(df_train_n[col].unique()) & set(df_test_n[col].unique()))
  df_train_n[col] = [x if x in list_rep else 'otro' for x in df_train_n[col]]
  df_test_n[col] = [x if x in list_rep else 'otro' for x in df_test_n[col]]
  model_col = smf.ols(formula=f'{TARGET}~{col}',data=df_train_n).fit()
  r_2_summary = round(model_col.rsquared, 3)
  r_2_train_cal = round(r2_score(df_train_n[TARGET], model_col.predict(df_train_n)),3)
  try:
    r_2_test_cal = round(r2_score(df_test_n[TARGET], model_col.predict(df_test_n)),3)
  except:
    r_2_test_cal = 'NC' # no calculado
  print(f'Modelo con sólo la variable {col}',
        f'tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo con sólo la variable ESTU_TIPODOCUMENTO tiene un R2 Resumen_modelo, train, test (0.057, 0.057, 0.056)
Modelo con sólo la variable COLE_AREA_UBICACION tiene un R2 Resumen_modelo, train, test (0.009, 0.009, 0.012)
Modelo con sólo la variable COLE_BILINGUE tiene un R2 Resumen_modelo, train, test (0.025, 0.025, 0.01)
Modelo con sólo la variable COLE_CALENDARIO tiene un R2 Resumen_modelo, train, test (0.06, 0.06, 0.048)
Modelo con sólo la variable COLE_CARACTER tiene un R2 Resumen_modelo, train, test (0.008, 0.008, 0.001)
Modelo con sólo la variable COLE_COD_DANE_ESTABLECIMIENTO tiene un R2 Resumen_modelo, train, test (0.213, 0.213, 0.164)
Modelo con sólo la variable COLE_COD_DANE_SEDE tiene un R2 Resumen_modelo, train, test (0.21, 0.21, 0.179)
Modelo con sólo la variable COLE_COD_DEPTO_UBICACION tiene un R2 Resumen_modelo, train, test (0.069, 0.069, 'NC')
Modelo con sólo la variable COLE_COD_MCPIO_UBICACION tiene un R2 Resumen_modelo, train, test (0.18, 0.18, 0.091)
Modelo con sólo 

In [76]:
val_counts = df_train_n[col].value_counts(1)
val_counts = val_counts[val_counts>0.05]
val_counts.index

Index(['11', '5', '76', '25', '8'], dtype='object')

En la parte de arriba hay casos en los que durante el train no se veía la categoría otro y al predecir sobre test si había categoría otro generando un error.
<br>
LLeve los elementos que estén en una categoria con baja frecuencia a una categoría otro

***En general categorias con frecuencia porcentual por debajo del 5% deberían agruparse dejandolas en una categoría de otros***

https://feature-engine.trainindata.com/en/0.6.x_a/encoders/RareLabelCategoricalEncoder.html

In [77]:
df_train_n = df_train.copy()
df_test_n = df_test.copy()
for col in list_vars_model:
  list_rep = list(set(df_train_n[col].unique()) & set(df_test_n[col].unique()))
  val_counts = df_train_n[col].value_counts(1)
  val_counts = val_counts[val_counts>0.05]  # si está por debajo del 5% iría a otro
  list_rep = [x for x in list_rep if x in val_counts.index]
  df_train_n[col] = [x if x in list_rep else 'otro' for x in df_train_n[col]]
  df_test_n[col] = [x if x in list_rep else 'otro' for x in df_test_n[col]]
  model_col = smf.ols(formula=f'{TARGET}~{col}',data=df_train_n).fit()
  r_2_summary = round(model_col.rsquared, 3)
  r_2_train_cal = round(r2_score(df_train_n[TARGET], model_col.predict(df_train_n)),3)

  r_2_test_cal = round(r2_score(df_test_n[TARGET], model_col.predict(df_test_n)),3)

  print(f'Modelo con sólo la variable {col}',
        f'tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo con sólo la variable ESTU_TIPODOCUMENTO tiene un R2 Resumen_modelo, train, test (0.053, 0.053, 0.055)
Modelo con sólo la variable COLE_AREA_UBICACION tiene un R2 Resumen_modelo, train, test (0.009, 0.009, 0.012)
Modelo con sólo la variable COLE_BILINGUE tiene un R2 Resumen_modelo, train, test (0.025, 0.025, 0.01)
Modelo con sólo la variable COLE_CALENDARIO tiene un R2 Resumen_modelo, train, test (0.042, 0.042, 0.018)
Modelo con sólo la variable COLE_CARACTER tiene un R2 Resumen_modelo, train, test (0.008, 0.008, 0.001)
Modelo con sólo la variable COLE_COD_DANE_ESTABLECIMIENTO tiene un R2 Resumen_modelo, train, test (0.0, 0.0, -0.0)
Modelo con sólo la variable COLE_COD_DANE_SEDE tiene un R2 Resumen_modelo, train, test (0.0, 0.0, -0.0)
Modelo con sólo la variable COLE_COD_DEPTO_UBICACION tiene un R2 Resumen_modelo, train, test (0.026, 0.026, 0.031)
Modelo con sólo la variable COLE_COD_MCPIO_UBICACION tiene un R2 Resumen_modelo, train, test (0.028, 0.028, 0.033)
Modelo con sólo la 

## Revisando categorias de variables antes y después de transformar

### Depto

In [80]:
df_train['COLE_COD_DEPTO_UBICACION'].value_counts(1)

11    0.172932
5     0.127820
76    0.085599
25    0.070272
8     0.061307
68    0.049451
13    0.040486
73    0.031810
52    0.030075
15    0.029497
23    0.028918
20    0.025448
19    0.024870
54    0.024581
50    0.024291
41    0.023135
47    0.023135
17    0.021110
66    0.019086
70    0.016484
63    0.014748
44    0.014170
27    0.010121
85    0.009543
18    0.007230
86    0.005205
81    0.004916
91    0.001446
95    0.001157
99    0.000868
88    0.000289
Name: COLE_COD_DEPTO_UBICACION, dtype: float64

In [79]:
df_train_n['COLE_COD_DEPTO_UBICACION'].value_counts()

otro    1667
11       598
5        442
76       296
25       243
8        212
Name: COLE_COD_DEPTO_UBICACION, dtype: int64

### cod dane establecimiento

In [81]:
df_train['COLE_COD_DANE_ESTABLECIMIENTO'].value_counts(1)

176834000092.0    0.002313
176001001753.0    0.002313
105615000236.0    0.002024
141016000305.0    0.002024
311001105863.0    0.002024
                    ...   
144430000511.0    0.000289
170771000013.0    0.000289
120310000014.0    0.000289
117042000561.0    0.000289
215104000092.0    0.000289
Name: COLE_COD_DANE_ESTABLECIMIENTO, Length: 2206, dtype: float64

In [82]:
df_train['COLE_COD_DANE_ESTABLECIMIENTO'].value_counts()

176834000092.0    8
176001001753.0    8
105615000236.0    7
141016000305.0    7
311001105863.0    7
                 ..
144430000511.0    1
170771000013.0    1
120310000014.0    1
117042000561.0    1
215104000092.0    1
Name: COLE_COD_DANE_ESTABLECIMIENTO, Length: 2206, dtype: int64

In [83]:
df_train_n['COLE_COD_DANE_ESTABLECIMIENTO'].value_counts()

otro    3458
Name: COLE_COD_DANE_ESTABLECIMIENTO, dtype: int64

**Si hay categorias con muy pocos valores intente agruparlas**

## Eliminando variables que no aporten al análisis

In [84]:
df_train_n = df_train.copy()
df_test_n = df_test.copy()
vars_candidatas = []
umbral_r2 = 0.0001
for col in list_vars_model:
  list_rep = list(set(df_train_n[col].unique()) & set(df_test_n[col].unique()))
  val_counts = df_train_n[col].value_counts(1)
  val_counts = val_counts[val_counts>0.05]  # si está por debajo del 5% iría a otro
  list_rep = [x for x in list_rep if x in val_counts.index]
  df_train_n[col] = [x if x in list_rep else 'otro' for x in df_train_n[col]]
  df_test_n[col] = [x if x in list_rep else 'otro' for x in df_test_n[col]]
  model_col = smf.ols(formula=f'{TARGET}~{col}',data=df_train_n).fit()
  r_2_summary = round(model_col.rsquared, 3)
  r_2_train_cal = round(r2_score(df_train_n[TARGET], model_col.predict(df_train_n)),3)

  r_2_test_cal = round(r2_score(df_test_n[TARGET], model_col.predict(df_test_n)),3)
  # agregando las columnas que tengan r2 mayor que umbral
  if r_2_train_cal>umbral_r2:
    vars_candidatas.append(col)
  print(f'Modelo con sólo la variable {col}',
        f'tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo con sólo la variable ESTU_TIPODOCUMENTO tiene un R2 Resumen_modelo, train, test (0.053, 0.053, 0.055)
Modelo con sólo la variable COLE_AREA_UBICACION tiene un R2 Resumen_modelo, train, test (0.009, 0.009, 0.012)
Modelo con sólo la variable COLE_BILINGUE tiene un R2 Resumen_modelo, train, test (0.025, 0.025, 0.01)
Modelo con sólo la variable COLE_CALENDARIO tiene un R2 Resumen_modelo, train, test (0.042, 0.042, 0.018)
Modelo con sólo la variable COLE_CARACTER tiene un R2 Resumen_modelo, train, test (0.008, 0.008, 0.001)
Modelo con sólo la variable COLE_COD_DANE_ESTABLECIMIENTO tiene un R2 Resumen_modelo, train, test (0.0, 0.0, -0.0)
Modelo con sólo la variable COLE_COD_DANE_SEDE tiene un R2 Resumen_modelo, train, test (0.0, 0.0, -0.0)
Modelo con sólo la variable COLE_COD_DEPTO_UBICACION tiene un R2 Resumen_modelo, train, test (0.026, 0.026, 0.031)
Modelo con sólo la variable COLE_COD_MCPIO_UBICACION tiene un R2 Resumen_modelo, train, test (0.028, 0.028, 0.033)
Modelo con sólo la 

In [86]:
# variables eliminadas
set(list_vars_model) - set(vars_candidatas)

{'COLE_CODIGO_ICFES',
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_COD_DANE_SEDE',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'COLE_NOMBRE_SEDE',
 'ESTU_FECHANACIMIENTO',
 'ESTU_PRIVADO_LIBERTAD'}

In [88]:
df_train['ESTU_PRIVADO_LIBERTAD'].value_counts()

N    3458
Name: ESTU_PRIVADO_LIBERTAD, dtype: int64

In [89]:
df_test['ESTU_PRIVADO_LIBERTAD'].value_counts()

N    1153
Name: ESTU_PRIVADO_LIBERTAD, dtype: int64

In [87]:
df_train['ESTU_FECHANACIMIENTO'].value_counts()

12/11/2002    9
05/09/2002    8
30/11/2002    7
10/12/2002    6
20/10/2002    6
             ..
22/12/2000    1
05/04/1994    1
08/09/2000    1
25/09/1998    1
19/12/2002    1
Name: ESTU_FECHANACIMIENTO, Length: 1966, dtype: int64

In [85]:
vars_candidatas

['ESTU_TIPODOCUMENTO',
 'COLE_AREA_UBICACION',
 'COLE_BILINGUE',
 'COLE_CALENDARIO',
 'COLE_CARACTER',
 'COLE_COD_DEPTO_UBICACION',
 'COLE_COD_MCPIO_UBICACION',
 'COLE_DEPTO_UBICACION',
 'COLE_GENERO',
 'COLE_JORNADA',
 'COLE_MCPIO_UBICACION',
 'COLE_NATURALEZA',
 'COLE_SEDE_PRINCIPAL',
 'ESTU_COD_DEPTO_PRESENTACION',
 'ESTU_COD_MCPIO_PRESENTACION',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_DEPTO_PRESENTACION',
 'ESTU_DEPTO_RESIDE',
 'ESTU_ESTADOINVESTIGACION',
 'ESTU_GENERO',
 'ESTU_MCPIO_PRESENTACION',
 'ESTU_MCPIO_RESIDE',
 'ESTU_NACIONALIDAD',
 'ESTU_PAIS_RESIDE',
 'FAMI_CUARTOSHOGAR',
 'FAMI_EDUCACIONMADRE',
 'FAMI_EDUCACIONPADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_PERSONASHOGAR',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENELAVADORA']

In [90]:
df_train.columns

Index(['ESTU_TIPODOCUMENTO', 'COLE_AREA_UBICACION', 'COLE_BILINGUE',
       'COLE_CALENDARIO', 'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO',
       'COLE_COD_DANE_SEDE', 'COLE_COD_DEPTO_UBICACION',
       'COLE_COD_MCPIO_UBICACION', 'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION',
       'COLE_GENERO', 'COLE_JORNADA', 'COLE_MCPIO_UBICACION',
       'COLE_NATURALEZA', 'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_FECHANACIMIENTO', 'ESTU_GENERO',
       'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE', 'ESTU_NACIONALIDAD',
       'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD', 'FAMI_CUARTOSHOGAR',
       'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE', 'FAMI_ESTRATOVIVIENDA',
       'FAMI_PERSONASHOGAR', 'FAMI_TIENEAUTOMOVIL', 'FAMI_TIENECOMPUTADOR',

## Creación de la variable edad

(retomo la construcción de los datasets train y test)

In [127]:
list_vars_select = vars_candidatas + ['ESTU_FECHANACIMIENTO', 'PERIODO', TARGET]

In [128]:
data_modelo_con_edad = data_saber[list_vars_select].dropna().copy()
df_train, df_test = train_test_split(data_modelo_con_edad, test_size=0.25, random_state=r_state)
len(df_train), len(df_test), len(df_train)/len(data_modelo_con_edad), len(df_test)/len(data_modelo_con_edad)

(3458, 1153, 0.7499457818260681, 0.2500542181739319)

## Preprocesamiento construcción edad

In [129]:
print(df_train['ESTU_FECHANACIMIENTO'].sample(1))
print(df_train['PERIODO'].sample(1))

1812    27/07/1996
Name: ESTU_FECHANACIMIENTO, dtype: object
7969    20152
Name: PERIODO, dtype: int64


In [130]:
[int(x[-4:]) for x in df_train['ESTU_FECHANACIMIENTO']]

[2002,
 2001,
 2001,
 1999,
 1999,
 1995,
 1997,
 1998,
 1999,
 2000,
 1998,
 1999,
 2000,
 2001,
 1999,
 2001,
 2001,
 2002,
 2002,
 1999,
 1996,
 2002,
 1998,
 1999,
 1997,
 2003,
 1998,
 1997,
 1999,
 1997,
 1999,
 2000,
 2002,
 2000,
 1999,
 1995,
 1996,
 2003,
 1998,
 2000,
 2002,
 2002,
 1997,
 2001,
 2000,
 1998,
 1999,
 1998,
 2000,
 1999,
 1996,
 2000,
 2002,
 1997,
 1999,
 2002,
 1997,
 2002,
 1999,
 2002,
 2000,
 2002,
 2002,
 2000,
 1999,
 2001,
 2000,
 2001,
 1997,
 2003,
 1996,
 2001,
 1998,
 2000,
 2002,
 1998,
 1999,
 2001,
 1998,
 1998,
 2000,
 2000,
 2002,
 1999,
 2003,
 1998,
 1997,
 1997,
 1997,
 2000,
 2003,
 1998,
 1999,
 2000,
 1999,
 1995,
 1997,
 2001,
 1999,
 2000,
 2000,
 2001,
 2002,
 1999,
 1998,
 2001,
 1996,
 1995,
 1998,
 1999,
 1982,
 1995,
 2003,
 2000,
 2003,
 1998,
 1990,
 1998,
 2003,
 1997,
 2001,
 1998,
 1998,
 2002,
 1998,
 2000,
 2001,
 1995,
 1987,
 2003,
 1997,
 1995,
 2001,
 2000,
 1999,
 2001,
 2001,
 2000,
 2002,
 1997,
 1998,
 1997,
 2002,

In [131]:
df_train['AÑO_NACIMIENTO'] = [int(x[-4:]) if int(x[-4:])>100 else int(x[-4:])+2000 for x in df_train['ESTU_FECHANACIMIENTO']]
df_train['AÑO_PRESENTACION'] = [int(str(x)[:4]) for x in df_train['PERIODO']]
df_train['EDAD'] = df_train['AÑO_PRESENTACION'] - df_train['AÑO_NACIMIENTO']
df_train[['AÑO_NACIMIENTO', 'ESTU_FECHANACIMIENTO', 'AÑO_PRESENTACION', 'PERIODO', 'EDAD']].sample(10)

,AÑO_NACIMIENTO,ESTU_FECHANACIMIENTO,AÑO_PRESENTACION,PERIODO,EDAD
501,2001,20/01/2001,2016,20162,15
7121,1998,14/01/1998,2014,20142,16
9956,1997,23/03/1997,2014,20142,17
7297,1999,13/02/1999,2016,20162,17
994,1998,27/01/1998,2014,20142,16
2259,1999,02/04/1999,2015,20152,16
5023,2002,29/10/2002,2019,20194,17
3592,1998,13/09/1998,2016,20162,18
2556,1999,26/06/1999,2016,20162,17
222,2001,05/10/2001,2019,20194,18


In [132]:
df_test['AÑO_NACIMIENTO'] = [int(x[-4:]) if int(x[-4:])>100 else int(x[-4:])+2000 for x in df_test['ESTU_FECHANACIMIENTO']]
df_test['AÑO_PRESENTACION'] = [int(str(x)[:4]) for x in df_test['PERIODO']]
df_test['EDAD'] = df_test['AÑO_PRESENTACION'] - df_test['AÑO_NACIMIENTO']
df_test[['AÑO_NACIMIENTO', 'ESTU_FECHANACIMIENTO', 'AÑO_PRESENTACION', 'PERIODO', 'EDAD']].sample(10)

,AÑO_NACIMIENTO,ESTU_FECHANACIMIENTO,AÑO_PRESENTACION,PERIODO,EDAD
5695,1999,16/06/1999,2017,20172,18
1769,1995,20/05/1995,2015,20152,20
6548,1999,11/04/1999,2015,20152,16
1041,1998,04/11/1998,2015,20152,17
1794,2001,20/11/2001,2019,20194,18
7264,2003,10/03/2003,2019,20194,16
1532,1997,06/09/1997,2017,20172,20
4724,2002,28/12/2002,2019,20194,17
8907,2000,08/01/2000,2017,20172,17
7857,2000,24/07/2000,2017,20172,17


In [133]:
df_train = df_train[(df_train['EDAD']>10)].copy()
df_train['EDAD'].describe()

count    3453.000000
mean       17.796119
std         3.138945
min        14.000000
25%        17.000000
50%        17.000000
75%        18.000000
max        56.000000
Name: EDAD, dtype: float64

In [134]:
px.bar(df_train['EDAD'].value_counts().reset_index().rename(columns={'index':'EDAD', 'EDAD':'CONTEO'}),
       x='EDAD', y='CONTEO')

In [135]:
px.bar(df_test['EDAD'].value_counts().reset_index().rename(columns={'index':'EDAD', 'EDAD':'CONTEO'}),
       x='EDAD', y='CONTEO')

## Preprocesamiento de las variables categoricas

In [136]:
for col in vars_candidatas:
  list_rep = list(set(df_train[col].unique()) & set(df_test[col].unique()))
  val_counts = df_train[col].value_counts(1)
  val_counts = val_counts[val_counts>0.05]  # si está por debajo del 5% iría a otro
  list_rep = [x for x in list_rep if x in val_counts.index]
  df_train[col] = [x if x in list_rep else 'otro' for x in df_train[col]]
  df_test[col] = [x if x in list_rep else 'otro' for x in df_test[col]]

In [137]:
df_train

,ESTU_TIPODOCUMENTO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_COD_DEPTO_UBICACION,COLE_COD_MCPIO_UBICACION,COLE_DEPTO_UBICACION,COLE_GENERO,COLE_JORNADA,...,FAMI_TIENEAUTOMOVIL,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,ESTU_FECHANACIMIENTO,PERIODO,PUNT_GLOBAL,AÑO_NACIMIENTO,AÑO_PRESENTACION,EDAD
4468,TI,RURAL,N,A,ACADÉMICO,otro,otro,otro,MIXTO,MAÑANA,...,No,No,Si,Si,10/12/2002,20194,246.0,2002,2019,17
1955,TI,URBANO,N,A,TÉCNICO/ACADÉMICO,otro,otro,otro,MIXTO,MAÑANA,...,No,No,No,Si,21/12/2001,20194,236.0,2001,2019,18
8976,TI,URBANO,N,A,TÉCNICO/ACADÉMICO,otro,otro,otro,MIXTO,MAÑANA,...,Si,No,No,Si,02/04/2001,20172,256.0,2001,2017,16
3711,TI,URBANO,N,A,TÉCNICO/ACADÉMICO,otro,otro,otro,MIXTO,MAÑANA,...,Si,Si,Si,Si,08/07/1999,20162,223.0,1999,2016,17
3473,TI,URBANO,N,A,TÉCNICO/ACADÉMICO,11,11001,BOGOTA,MIXTO,COMPLETA,...,No,Si,Si,Si,06/10/1999,20162,280.0,1999,2016,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8391,TI,URBANO,N,A,TÉCNICO/ACADÉMICO,otro,otro,otro,MIXTO,MAÑANA,...,No,No,No,No,17/07/2000,20172,232.0,2000,2017,17
5841,TI,URBANO,N,A,TÉCNICO,76,otro,VALLE,MIXTO,MAÑANA,...,No,No,Si,Si,15/12/2000,20172,275.0,2000,2017,17
6014,TI,URBANO,N,A,TÉCNICO,otro,otro,otro,MIXTO,MAÑANA,...,Si,Si,No,Si,10/04/1998,20152,259.0,1998,2015,17
8083,TI,RURAL,otro,otro,ACADÉMICO,5,otro,ANTIOQUIA,MIXTO,COMPLETA,...,Si,Si,Si,Si,20/03/2000,20181,366.0,2000,2018,18


## Modelo 1

'ESTU_TIPODOCUMENTO + COLE_AREA_UBICACION + COLE_BILINGUE + COLE_CALENDARIO + COLE_CARACTER + COLE_DEPTO_UBICACION + COLE_GENERO + COLE_JORNADA + COLE_MCPIO_UBICACION + COLE_NATURALEZA + COLE_SEDE_PRINCIPAL + ESTU_DEPTO_PRESENTACION + ESTU_DEPTO_RESIDE + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + ESTU_MCPIO_PRESENTACION + ESTU_MCPIO_RESIDE + ESTU_NACIONALIDAD + ESTU_PAIS_RESIDE + FAMI_CUARTOSHOGAR + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_PERSONASHOGAR + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + FAMI_TIENEINTERNET + FAMI_TIENELAVADORA + EDAD'

In [147]:
formula_1 = f'{TARGET} ~ ' + ' + '.join([x for x in vars_candidatas if '_COD_' not in x]) + ' + EDAD'
formula_1

'PUNT_GLOBAL ~ ESTU_TIPODOCUMENTO + COLE_AREA_UBICACION + COLE_BILINGUE + COLE_CALENDARIO + COLE_CARACTER + COLE_DEPTO_UBICACION + COLE_GENERO + COLE_JORNADA + COLE_MCPIO_UBICACION + COLE_NATURALEZA + COLE_SEDE_PRINCIPAL + ESTU_DEPTO_PRESENTACION + ESTU_DEPTO_RESIDE + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + ESTU_MCPIO_PRESENTACION + ESTU_MCPIO_RESIDE + ESTU_NACIONALIDAD + ESTU_PAIS_RESIDE + FAMI_CUARTOSHOGAR + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_PERSONASHOGAR + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + FAMI_TIENEINTERNET + FAMI_TIENELAVADORA + EDAD'

In [149]:
model_1 = smf.ols(formula=formula_1, data=df_train).fit()
r_2_summary = round(model_1.rsquared, 3)
r_2_train_cal = round(r2_score(df_train[TARGET], model_1.predict(df_train)),3)

r_2_test_cal = round(r2_score(df_test[TARGET], model_1.predict(df_test)),3)

print(f'Modelo tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo tiene un R2 Resumen_modelo, train, test (0.313, 0.313, 0.279)


In [155]:
df_train['COLE_MCPIO_UBICACION'].value_counts()

otro           2682
BOGOTÁ D.C.     598
CALI            173
Name: COLE_MCPIO_UBICACION, dtype: int64

In [153]:
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       0.313
Model:                            OLS   Adj. R-squared:                  0.299
Method:                 Least Squares   F-statistic:                     22.62
Date:                Thu, 31 Aug 2023   Prob (F-statistic):          5.59e-223
Time:                        01:36:45   Log-Likelihood:                -17689.
No. Observations:                3453   AIC:                         3.552e+04
Df Residuals:                    3384   BIC:                         3.594e+04
Df Model:                          68                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

## Modelo 2 (quitando info geo)

In [159]:
formula_2 = f'{TARGET} ~ '\
 + ' + '.join([x for x in vars_candidatas if ('_COD_' not in x) and ('_MCPIO_' not in x) and ('_RESIDE' not in x) and ('_UBICACION' not in x)]) + ' + EDAD'
formula_2

'PUNT_GLOBAL ~ ESTU_TIPODOCUMENTO + COLE_BILINGUE + COLE_CALENDARIO + COLE_CARACTER + COLE_GENERO + COLE_JORNADA + COLE_NATURALEZA + COLE_SEDE_PRINCIPAL + ESTU_DEPTO_PRESENTACION + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + ESTU_NACIONALIDAD + FAMI_CUARTOSHOGAR + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_PERSONASHOGAR + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + FAMI_TIENEINTERNET + FAMI_TIENELAVADORA + EDAD'

In [161]:
model_2 = smf.ols(formula=formula_2, data=df_train).fit()
r_2_summary = round(model_2.rsquared, 3)
r_2_train_cal = round(r2_score(df_train[TARGET], model_2.predict(df_train)),3)

r_2_test_cal = round(r2_score(df_test[TARGET], model_2.predict(df_test)),3)

print(f'Modelo tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo tiene un R2 Resumen_modelo, train, test (0.307, 0.307, 0.273)


In [171]:
df_train['ESTU_GENERO'].value_counts()

F    1912
M    1541
Name: ESTU_GENERO, dtype: int64

In [162]:
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     29.60
Date:                Thu, 31 Aug 2023   Prob (F-statistic):          8.22e-230
Time:                        01:47:20   Log-Likelihood:                -17702.
No. Observations:                3453   AIC:                         3.551e+04
Df Residuals:                    3401   BIC:                         3.583e+04
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

In [176]:
df_train['FAMI_PERSONASHOGAR'].value_counts(1)

3 a 4     0.233999
otro      0.173762
Cuatro    0.167101
5 a 6     0.149146
Cinco     0.123081
Tres      0.088619
Seis      0.064292
Name: FAMI_PERSONASHOGAR, dtype: float64

In [177]:
cols_excluir = ['FAMI_TIENELAVADORA',  'COLE_SEDE_PRINCIPAL',  'ESTU_NACIONALIDAD',
'COLE_NATURALEZA',  'COLE_CARACTER',  'FAMI_CUARTOSHOGAR',  'FAMI_PERSONASHOGAR',   'FAMI_TIENEINTERNET']

## modelo 3

In [179]:
formula_3 = f'{TARGET} ~ '\
 + ' + '.join([x for x in vars_candidatas if ('_COD_' not in x) and
  ('_MCPIO_' not in x) and ('_RESIDE' not in x) and ('_UBICACION' not in x) and (x not in cols_excluir)]) + ' + EDAD'
formula_3

'PUNT_GLOBAL ~ ESTU_TIPODOCUMENTO + COLE_BILINGUE + COLE_CALENDARIO + COLE_GENERO + COLE_JORNADA + ESTU_DEPTO_PRESENTACION + ESTU_ESTADOINVESTIGACION + ESTU_GENERO + FAMI_EDUCACIONMADRE + FAMI_EDUCACIONPADRE + FAMI_ESTRATOVIVIENDA + FAMI_TIENEAUTOMOVIL + FAMI_TIENECOMPUTADOR + EDAD'

In [180]:
model_3 = smf.ols(formula=formula_3, data=df_train).fit()
r_2_summary = round(model_3.rsquared, 3)
r_2_train_cal = round(r2_score(df_train[TARGET], model_3.predict(df_train)),3)

r_2_test_cal = round(r2_score(df_test[TARGET], model_3.predict(df_test)),3)

print(f'Modelo tiene un R2 Resumen_modelo, train, test {r_2_summary, r_2_train_cal, r_2_test_cal}')

Modelo tiene un R2 Resumen_modelo, train, test (0.293, 0.293, 0.265)


In [186]:
df_train['ESTU_ESTADOINVESTIGACION'].value_counts()

PUBLICAR    3452
otro           1
Name: ESTU_ESTADOINVESTIGACION, dtype: int64

In [181]:
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       0.293
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     41.61
Date:                Thu, 31 Aug 2023   Prob (F-statistic):          7.23e-228
Time:                        02:04:53   Log-Likelihood:                -17738.
No. Observations:                3453   AIC:                         3.555e+04
Df Residuals:                    3418   BIC:                         3.576e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

**RETO**

El modelo nos indica que por cada año de edad del aspirante el puntaje global baja alredor de un punto <br>

pruebe transformar aplicando log al target y a la variable edad
<br>
saque la variable ESTU_ESTADOINVESTIGACION

<br>
Recategorice las vars FAMI_EDUCACIONMADRE y FAMI_EDUCACIONPADRE y cree interacciones

<br>

Revise asociacón de la varaible estrato con tenencia de automovil y de computador


In [182]:
px.bar(df_train['EDAD'].value_counts().reset_index().rename(columns={'index':'EDAD', 'EDAD':'CONTEO'}),
       x='EDAD', y='CONTEO')

In [184]:
px.histogram(df_train[TARGET])